In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.1.0
    Uninstalling charset-normalizer-3.1.0:
      Successfully uninstalled charset-normalizer-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trafilatura 1.5.0 requires charset-normalizer>=3.1.0; python_version >= "3.7", but you have charset-normalizer 2.0.12 which is incompatible.
htmldate 1.4.2 requires charset-normalizer>=3.1.0; python_version >= "3.7", but you have charset-normalizer 2.0.12 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install trafilatura 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached charset_normalizer-3.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (199 kB)
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 2.0.12
    Uninstalling charset-normalizer-2.0.12:
      Successfully uninstalled charset-normalizer-2.0.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.27.1 requires charset-normalizer~=2.0.0; python_version >= "3", but you have charset-normalizer 3.1.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModel

#model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_ckpt = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
     
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]
  
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
import pandas as pd

df=pd.read_csv('Test Links.csv')
df=df['Link']

Loading Dataset for getting just the title.

In [ ]:
import pandas as pd

title_dataset=pd.read_csv('ExtractedText.csv')
title_dataset=title_dataset[['DocumentID','Title']]
final_title_dataset = pd.DataFrame(columns=['DocID','Title'])

In [ ]:

# drop duplicates and create new dataframe
title_dataset = title_dataset.drop_duplicates(subset=['DocumentID', 'Title'])

print(title_dataset[title_dataset['DocumentID']==4]['Title'].values[0])

Ending Homework Battles | Psychology Today


In [ ]:
df.shape[0]

200

In [ ]:
from datasets import load_dataset,Dataset

doc_Df = pd.DataFrame(columns=['Title','DocID','URL','Embeddings'])
Dataset_doc_Df=Dataset.from_pandas(doc_Df)

In [ ]:
print(doc_Df)

Empty DataFrame
Columns: [Title, DocID, URL, Embeddings]
Index: []


In [ ]:
import trafilatura
# url='https://www.psychologytoday.com/us/basics/child-development'

for i,url in enumerate(df):
  t=trafilatura.fetch_url(url)
  text=trafilatura.extract(t)
  # doc_Df['DocID'][i]=i
  # doc_Df['URL'][i]=link
  # doc_Df['Embeddings'][i]=get_embeddings(text)
  doc_Df.loc[i]=[title_dataset[title_dataset['DocumentID']==i]['Title'].values[0],i,url,get_embeddings(text).detach().cpu().numpy()[0]]


In [ ]:
doc_Df

,Title,DocID,URL,Embeddings
0,Child Development | Psychology Today,0,https://www.mother.ly/life/motherly-stories/ki...,"[0.2803007, 0.23809318, -0.19282328, -0.159848..."
1,Parenting a Child with ADHD | Psychology Today,1,https://www.mother.ly/life/motherly-stories/ki...,"[-0.057591643, 0.2632406, 0.36278155, 0.245281..."
2,Raising an Adopted Child | Psychology Today,2,https://www.mother.ly/life/motherly-stories/ki...,"[0.30209792, -0.043051302, -0.1418182, -0.2052..."
3,Parenting a Child with Autism | Psychology Today,3,https://www.mother.ly/life/motherly-stories/ki...,"[-0.030700075, 0.390369, 0.41788584, -0.278411..."
4,Ending Homework Battles | Psychology Today,4,https://www.mother.ly/life/motherly-stories/ki...,"[0.26746687, 0.18432264, -0.11905984, 0.089509..."
...,...,...,...,...
195,My Kid's Toddler Tantrums Are Getting Out of H...,195,https://www.mother.ly/life/motherly-stories/ki...,"[0.17019387, -0.038686477, 0.18198003, -0.1769..."
196,10 Ways to Make Feeding Your Toddler 10 Times ...,196,https://www.mother.ly/life/motherly-stories/ki...,"[0.4790843, 0.36725512, -0.06070934, -0.125382..."
197,Separation and Social Anxiety in Kids,197,https://www.mother.ly/life/motherly-stories/ki...,"[0.097834304, -0.50916183, 0.33884814, -0.6288..."
198,How Much Does My Kid Need to Eat?,198,https://www.mother.ly/life/motherly-stories/ki...,"[0.9504991, 0.4124903, 0.634642, -0.18082817, ..."


In [ ]:
doc_Df.to_csv('CompleteMainText.csv')

In [ ]:
embeddings_dataset = Dataset.from_pandas(doc_Df)
embeddings_dataset=embeddings_dataset.remove_columns('__index_level_0__')
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings'],
    num_rows: 200
})

In [ ]:
embeddings_dataset.add_faiss_index(column="Embeddings")


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings'],
    num_rows: 200
})

In [ ]:
# loading library
import pickle
# create an iterator object with write permission - model.pkl
with open('Distillbert_DocumentLevel_pkl', 'wb') as files:
    pickle.dump(embeddings_dataset, files)

# We will save/send this file to the Data Base.